### Select patients for CFR model: Split patients in train, val and test sets ###

In [2]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [3]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
cfr_meta_date = '200219'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+cfr_meta_date)
print(meta_dir)

/mnt/obi0/andreas/data/cfr/metadata_200219


In [5]:
match_view_filename = 'BWH_2020-Feb-19_pet_match365_diff_files_'+cfr_meta_date+'.parquet'
files_cfr = pd.read_parquet(os.path.join(meta_dir, match_view_filename))

print('Total number of patients {}'.format(len(files_cfr.mrn.unique())))
print('Total number of studies  {}'.format(len(files_cfr.study.unique())))
print('Total number of files    {}'.format(len(files_cfr.filename.unique())))

files_cfr.head()

Total number of patients 1626
Total number of studies  2751
Total number of files    126145


,study,mrn,echo_study_date,mrnstudydateid,days_post_cfr,reportID,subjectID,cfr_study_date,Reason_post_MI_eval,Reason_post_CABG,Reason_post_PCI,Reason_myocardial_viability,reason_postHeartTransplant,hx_recent_MI_yes,troponin,hx_prior_MI_yes,hx_prior_transplant,hx_prior_transplant_yes,hx_CABG,hx_CABG_yes,hx_PCI,hx_PCI_yes,hx_CHF,hx_CHF_yes,hx_cardiomyopathy_yes,...,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view
0,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2435,191,162094.0,7817.0,2018-04-25,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,2015-01-01,True,NaT,False,False,...,2.565391e-08,7.528357e-13,4.774701e-10,2.708740e-12,3.670912e-09,1.567517e-11,1.610227e-10,1.509277e-07,2.566710e-06,1.363240e-13,2.349999e-08,7.267388e-09,2.949915e-12,9.663496e-15,1.294855e-04,9.732323e-06,1.612119e-12,9.998580e-01,4.653681e-10,5.331582e-11,3.434301e-11,4.860822e-09,5.479821e-14,2.986510e-08,psax_avz
1,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2435,191,162094.0,7817.0,2018-04-25,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,2015-01-01,True,NaT,False,False,...,1.450411e-07,1.366147e-09,3.045583e-11,2.590847e-12,3.514123e-09,1.119247e-12,1.174291e-11,1.653268e-09,6.647998e-09,7.400897e-07,4.554047e-08,2.202479e-12,1.033245e-11,1.524424e-10,1.032910e-10,1.392449e-09,8.828201e-10,5.749271e-07,9.999073e-01,7.699048e-07,6.217265e-08,8.135313e-09,1.209222e-09,9.039690e-05,psax_az
2,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2435,191,162094.0,7817.0,2018-04-25,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,2015-01-01,True,NaT,False,False,...,2.660267e-07,3.196280e-10,2.934359e-09,3.283927e-07,3.254719e-09,1.172012e-07,4.847203e-10,3.966162e-09,1.291069e-09,1.924162e-08,6.683372e-07,9.999976e-01,1.350526e-09,4.165070e-10,1.177100e-10,1.866018e-09,3.144208e-09,3.432988e-10,9.421681e-10,8.573890e-07,4.335133e-08,1.434178e-10,4.363453e-12,2.205031e-09,other
3,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2435,191,162094.0,7817.0,2018-04-25,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,2015-01-01,True,NaT,False,False,...,1.941936e-14,2.563091e-18,2.315560e-14,1.126485e-08,5.550321e-15,1.119701e-19,4.695120e-16,9.429649e-14,1.808776e-17,1.098487e-13,1.432877e-15,2.962554e-15,2.005398e-15,3.412526e-12,1.698335e-14,8.875975e-14,8.122525e-18,1.691228e-16,5.149206e-17,7.832482e-14,1.353168e-15,1.369931e-19,3.447312e-22,2.979074e-17,a2c
4,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2435,191,162094.0,7817.0,2018-04-25,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,2015-01-01,True,NaT,False,False,...,4.836035e-07,9.999956e-01,4.835767e-11,1.211003e-13,1.254464e-09,3.868239e-12,6.760433e-14,1.656497e-10,1.586072e-09,2.450811e-11,3.360560e-12,2.185993e-14,4.238282e-11,6.147999e-12,9.961027e-12,3.035175e-11,3.153882e-10,3.031236e-14,4.888586e-13,3.554045e-12,1.007157e-11,7.237515e-15,2.361346e-18,9.150120e-12,a3c_laocc


### Split the patients in train, validate and test sets ###
Although each view might have a little different patient pupulation distribution, because not all views are in each study. However, we want the same MRNS in each data set and for all views so that we can directly compare the performance of the algorithm for the same patients. We can expand the data frame above to add the splits.

In [6]:
def patientsplit(patient_list):

    train_test_split = 0.80
    train_eval_split = 0.90

    # Take a test set from all patients
    patient_list_train = np.random.choice(patient_list,
                                          size = int(np.floor(train_test_split*len(patient_list))),
                                          replace = False)
    patient_list_test = list(set(patient_list).difference(patient_list_train))
    train_test_intersection = set(patient_list_train).intersection(set(patient_list_test)) # This should be empty
    print('Intersection of patient_list_train and patient_list_test:', train_test_intersection)

    # Further separate some patients for evaluation from the training list
    patient_list_eval = np.random.choice(patient_list_train,
                                         size = int(np.ceil((1-train_eval_split)*len(patient_list_train))),
                                         replace = False)

    patient_list_train = set(patient_list_train).difference(patient_list_eval)
    train_eval_intersection = set(patient_list_train).intersection(set(patient_list_eval))
    print('Intersection of patient_list_train and patient_list_eval:', train_eval_intersection)

    # Show the numbers
    print('total patients:', len(patient_list))
    print()
    print('patients in set:', np.sum([len(patient_list_train),
                                     len(patient_list_eval),
                                     len(patient_list_test)]))
    print()
    print('patients in train:', len(patient_list_train))
    print('patients in eval:', len(patient_list_eval))
    print('patients in test:', len(patient_list_test))

    return patient_list_train, patient_list_eval, patient_list_test

In [7]:
# Get a patient list
patient_list = list(files_cfr.sample(frac=1).mrn.unique())
patient_list_train, patient_list_eval, patient_list_test = patientsplit(patient_list)

patient_split = {'train': patient_list_train,
                 'eval': patient_list_eval,
                 'test': patient_list_test}

print('Patient IDs in train:', len(patient_split['train']))
print('Patient IDs in eval:', len(patient_split['eval']))
print('Patient IDs in test:', len(patient_split['test']))

print()

print('Intersection train-test:', set(patient_split['train']).intersection(set(patient_split['test'])))
print('Intersection train-eval:', set(patient_split['train']).intersection(set(patient_split['eval'])))
print('Intersection eval-test:', set(patient_split['eval']).intersection(set(patient_split['test'])))

Intersection of patient_list_train and patient_list_test: set()
Intersection of patient_list_train and patient_list_eval: set()
total patients: 1626

patients in set: 1626

patients in train: 1170
patients in eval: 130
patients in test: 326
Patient IDs in train: 1170
Patient IDs in eval: 130
Patient IDs in test: 326

Intersection train-test: set()
Intersection train-eval: set()
Intersection eval-test: set()


In [11]:
# Add mode column to files_cfr
split_list = []
for dset in patient_split.keys():
    dset_mrn_list = list(patient_split[dset])
    split_list.append(pd.DataFrame({'mrn': dset_mrn_list,
                                    'mode': [dset]*len(dset_mrn_list)}))

split_df = pd.concat(split_list, ignore_index = True)

files_cfr_dset = files_cfr.merge(right = split_df, on = 'mrn', how = 'left').\
                    sample(frac = 1).\
                    reset_index(drop = True)

In [12]:
files_cfr_dset.head(5)

,study,mrn,echo_study_date,mrnstudydateid,days_post_cfr,reportID,subjectID,cfr_study_date,Reason_post_MI_eval,Reason_post_CABG,Reason_post_PCI,Reason_myocardial_viability,reason_postHeartTransplant,hx_recent_MI_yes,troponin,hx_prior_MI_yes,hx_prior_transplant,hx_prior_transplant_yes,hx_CABG,hx_CABG_yes,hx_PCI,hx_PCI_yes,hx_CHF,hx_CHF_yes,hx_cardiomyopathy_yes,...,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view,mode
0,4a12b16cef154609_4903a58051d4fa002b81a7e66ba5,14099162,2015-10-30,1074,161,139407.0,6106.0,2015-05-22,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,NaT,False,NaT,False,False,...,4.537317e-10,5.804002e-07,3.158367e-12,1.905105e-08,2.156175e-11,2.273954e-08,2.211763e-07,2.343351e-08,3.845132e-08,4.766685e-09,3.489811e-08,3.560713e-08,2.068707e-07,5.778625e-12,6.139981e-06,5.789256e-06,4.339181e-09,6.428265e-08,8.776662e-04,2.988672e-04,1.884261e-02,8.964010e-06,9.794475e-01,suprasternal,eval
1,4a13436501c7629d_4903a58051d4f8993b4e808b0d64,15035850,2015-10-19,1153,368,134935.0,5639.0,2014-10-16,False,False,False,False,False,False,NaN,False,NaT,False,NaT,False,NaT,False,NaT,False,False,...,4.777956e-10,1.866401e-11,3.297864e-08,1.238187e-10,2.417305e-13,1.282559e-11,1.074862e-09,5.310468e-10,1.001931e-10,8.806489e-12,9.999998e-01,9.889346e-11,1.974058e-08,3.157220e-12,1.969700e-11,8.605273e-09,7.767808e-14,4.782155e-08,2.862419e-09,4.336790e-11,1.443152e-13,1.540048e-12,4.678454e-09,other,train
2,490af08d11319d9d_4903a581a728de0749dfcd83bfcc,29120938,2014-03-07,2326,-364,137693.0,6224.0,2015-03-06,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,NaT,False,2014-12-05,True,False,...,3.775941e-10,1.257427e-08,7.202875e-09,4.071412e-10,6.727767e-12,9.290200e-10,1.079158e-10,7.576385e-13,7.305218e-09,1.671548e-10,6.447029e-13,6.336469e-09,4.411319e-08,3.326450e-11,1.247998e-10,2.500883e-09,2.330092e-09,7.957955e-12,2.935514e-04,2.959193e-11,6.740934e-08,1.706691e-12,1.590862e-12,a2c,train
3,4b79c94086090f25_4903a5864e60c48a15389327c93d,2415545,2013-06-24,184,264,119823.0,5124.0,2012-10-03,False,False,False,False,False,False,0.0,True,NaT,False,NaT,False,2000-04-13,True,NaT,True,False,...,8.309269e-13,4.487829e-12,1.813532e-14,1.701018e-11,2.012539e-17,1.752380e-13,1.029398e-11,2.274827e-13,6.886560e-10,6.970119e-13,1.970913e-08,6.770214e-15,7.626497e-13,5.016595e-14,8.295456e-14,4.659224e-11,7.708144e-13,1.422705e-12,5.731794e-08,1.000000e+00,3.367883e-12,1.618232e-16,3.823100e-13,psax_pap,test
4,4b7f0b1646822ba0_4903a58793b67e3f91bb00a52644,4788790,2012-01-04,414,280,109280.0,3771.0,2011-03-30,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,NaT,False,NaT,False,False,...,4.726003e-11,4.061881e-14,4.927932e-11,2.153878e-13,9.942488e-16,1.282335e-14,8.800349e-11,4.675805e-13,2.270115e-14,5.210480e-13,1.000000e+00,1.209086e-16,6.535668e-13,1.037412e-13,1.173642e-13,8.032694e-13,4.041864e-14,6.735565e-12,5.478232e-12,1.260886e-09,1.030435e-12,2.957074e-14,2.717661e-14,other,train


In [13]:
# Prepare the final set that we will use for writing TFR files. We don't want any rows with NAs in some columns.
no_na_columns = ['study', 'mrn', 'echo_study_date', 'mrnstudydateid', 'model_cfr']
files_tfr = files_cfr_dset.dropna(subset=no_na_columns)

print('Dropped {} rows.'.format(files_cfr_dset.shape[0]-files_tfr.shape[0]))
# Add some other information that we need and shuffle the whole thing
files_tfr = files_tfr.assign(rate = np.round(1/files_tfr.frame_time*1e3, decimals = 1))
files_tfr = files_tfr.sample(frac = 1)

Dropped 0 rows.


In [14]:
dset_filename = 'tfr_files_dset_BWH_'+cfr_meta_date+'.parquet'
files_tfr.to_parquet(os.path.join(meta_dir, dset_filename))
print(files_tfr.shape)
files_tfr.head()

(126145, 403)


,study,mrn,echo_study_date,mrnstudydateid,days_post_cfr,reportID,subjectID,cfr_study_date,Reason_post_MI_eval,Reason_post_CABG,Reason_post_PCI,Reason_myocardial_viability,reason_postHeartTransplant,hx_recent_MI_yes,troponin,hx_prior_MI_yes,hx_prior_transplant,hx_prior_transplant_yes,hx_CABG,hx_CABG_yes,hx_PCI,hx_PCI_yes,hx_CHF,hx_CHF_yes,hx_cardiomyopathy_yes,...,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view,mode,rate
89371,4b7a8654cbbd514f_4903a5864f039a416d89d895f601,1123314,2013-11-26,60,1148,105973.0,3686.0,2010-10-05,False,False,False,False,False,False,0.0,True,NaT,False,NaT,False,1998-01-01,True,NaT,False,False,...,1.351767e-11,4.419790e-10,2.448445e-10,7.231782e-14,5.036059e-13,3.925761e-13,4.714285e-13,1.585348e-07,4.950185e-10,8.335611e-11,6.253902e-12,4.135860e-10,4.283221e-09,9.865905e-12,3.530055e-13,2.936099e-11,9.922407e-12,3.942255e-09,1.802185e-09,1.664403e-10,3.528703e-12,7.943476e-12,a2c,train,30.0
42701,4907bff27d5e17dc_4903a5858429801f3d2de72003c6,24671760,2010-07-09,2047,44,103443.0,310.0,2010-05-26,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,NaT,False,NaT,False,False,...,2.626884e-09,4.254108e-08,1.238194e-11,6.705236e-12,5.511274e-09,8.836610e-06,5.724635e-07,1.315104e-03,7.590859e-08,1.025470e-04,1.441149e-10,6.139988e-08,2.332140e-08,9.728783e-12,5.958461e-07,2.217573e-07,3.402669e-04,1.485731e-07,6.686805e-07,4.321768e-06,9.982257e-01,2.789302e-07,subcostal,train,30.0
88630,4b78d613bb26445f_4903a44ab12c9a22823d39680ad6,3395522,2009-07-16,284,93,94569.0,1856.0,2009-04-14,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,NaT,False,2009-04-10,True,False,...,2.713674e-10,2.894014e-13,9.365592e-10,1.902819e-10,4.844817e-12,1.693431e-11,7.578088e-12,1.510222e-11,1.197276e-05,6.375783e-15,1.791781e-12,1.910055e-10,1.382365e-11,1.479084e-11,6.661560e-10,7.261882e-05,2.546907e-11,2.130154e-02,9.786052e-01,7.932969e-09,5.057182e-16,4.628214e-12,psax_pap,train,29.8
112442,490170812e170e6e_4903a444c238fc20f454a23386d7,22537617,2007-12-27,1789,-4,84759.0,2018.0,2007-12-31,True,False,False,False,False,False,0.0,False,NaT,False,NaT,False,NaT,False,NaT,False,True,...,1.188330e-07,1.574487e-10,2.028785e-07,4.846819e-11,8.383523e-10,8.641399e-10,2.437283e-10,9.296557e-11,2.743214e-06,1.698678e-07,9.537925e-09,1.004291e-06,2.586413e-09,7.842194e-10,1.701850e-08,1.451032e-05,6.106723e-11,9.392062e-01,6.076623e-02,1.174224e-08,5.122733e-13,4.289182e-11,psax_mv,train,24.6
33088,4a12b7a1123149f2_4903a58051d6ea3df73e58756b51,14627152,2015-12-23,1123,-707,158876.0,2072.0,2017-11-29,True,False,False,False,False,False,0.0,True,NaT,False,NaT,False,NaT,False,NaT,False,False,...,9.489038e-15,6.395742e-12,9.503883e-13,7.059656e-13,4.143152e-12,3.134131e-13,2.984043e-11,1.992476e-12,5.511860e-14,1.000000e+00,1.948742e-13,1.519732e-14,2.302512e-13,1.797089e-11,7.230679e-14,7.067402e-17,5.215919e-13,1.462070e-13,4.074947e-11,8.079890e-15,6.948851e-14,2.759954e-13,other,train,23.0


In [15]:
minrate = 20
minframes = 40
minduration = minframes/minrate
print('Minmum rate:      {}'.format(minrate))
print('Minimum frames:   {}'.format(minframes))
print('Minimum duration: {}'.format(minduration))
#minduration = 1.5
#minframes = int(np.ceil(minrate*minduration))

maxrows = files_tfr.shape[0]
rows_disqualified = files_tfr[(files_tfr.rate < minrate) | (files_tfr.number_of_frames<minframes)].shape[0]
rows_qualified = files_tfr[(files_tfr.rate >= minrate) & (files_tfr.number_of_frames>=minframes)].shape[0]

print('Disqualified Rows {}, fraction:{:.1f}'.format(rows_disqualified, 
                                                     np.round(rows_disqualified/maxrows*100), decimals=1))

print('Qualified Rows {}, fraction:{:.1f}'.format(rows_qualified, 
                                                     np.round(rows_qualified/maxrows*100), decimals=1))

Minmum rate:      20
Minimum frames:   40
Minimum duration: 2.0
Disqualified Rows 33625, fraction:27.0
Qualified Rows 92519, fraction:73.0
